In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline

In [3]:
sns.set_context('notebook')
sns.set_style('darkgrid')

In [4]:
dataset_x = pd.read_csv("params.csv", index_col = 0)
dataset_y = pd.read_csv("slo.csv", index_col = 0)
all_dataset = pd.concat([dataset_x, dataset_y],axis=1)
dataset_x['iops'] = dataset_y['mixgraph_iops']

In [5]:
dataset_x.sort_values(by='iops',ascending = False).head(10)

,max_background_compactions,max_background_flushes,write_buffer_size,max_write_buffer_number,min_write_buffer_number_to_merge,max_bytes_for_level_multiplier,block_size,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,level0_stop_writes_trigger,iops
164,120,4,130000000,46,2,9.792984,318021,99,920,1024,108348.0
58,256,10,130000000,11,2,15.000000,29730,229,1,718,108337.0
2,256,10,130000000,10,2,15.000000,36665,230,1,664,108335.0
0,256,10,130000000,11,2,15.000000,29548,229,1,717,108323.0
84,256,10,130000000,11,2,15.000000,29983,229,1,718,108319.0
11,256,10,130000000,11,2,15.000000,30850,229,1,715,108229.0
41,256,10,130000000,15,2,15.000000,48954,225,66,670,108204.0
4,256,10,130000000,11,2,15.000000,30630,229,1,716,108185.0
94,256,10,130000000,11,2,15.000000,30751,229,1,713,108176.0
33,256,10,130000000,11,2,15.000000,32937,229,1,688,108171.0


In [ ]:
feature_corr = all_dataset.corr()
cor_target = abs(feature_corr["mixgraph_iops"])#Selecting highly correlated features
mixgraph_iops_corr = cor_target[cor_target>0.5]

In [7]:
def list_relevant_to_iops(features: list[str]) -> list[str]:
    """ Return a list of features from input features that are relevant to iops """
    result = []
    for f in features:
        if f in mixgraph_iops_corr:
            result.append(f)
    return result

In [42]:
def param_to_all_relevant_features_with_impact(param: str) -> list[str]:
    feature_corr = all_dataset.corr()
    cor_target = abs(feature_corr[param])#Selecting highly correlated features
    relevant_features = cor_target[cor_target>0.5]
    return relevant_features.keys() #list_relevant_to_iops(relevant_features.keys().tolist())

investigate_var = "rocksdb_bytes_per_read_count"
param_to_all_relevant_features_with_impact(investigate_var)


Index(['write_buffer_size', 'mem_p99', 'mixgraph_latency', 'mixgraph_iops',
       'mixgraph_io_size', 'rocksdb_memtable_miss_count',
       'rocksdb_number_keys_written_count', 'rocksdb_number_keys_read_count',
       'rocksdb_bytes_written_count', 'rocksdb_number_db_seek_count',
       'rocksdb_number_db_next_count', 'rocksdb_number_db_seek_found_count',
       'rocksdb_number_db_next_found_count',
       'rocksdb_db_iter_bytes_read_count', 'rocksdb_wal_bytes_count',
       'rocksdb_write_self_count', 'rocksdb_write_wal_count',
       'rocksdb_number_iter_skip_count', 'rocksdb_num_iterator_created_count',
       'rocksdb_num_iterator_deleted_count', 'rocksdb_db_get_micros_p50',
       'rocksdb_db_get_micros_count', 'rocksdb_db_get_micros_sum',
       'rocksdb_db_write_micros_p50', 'rocksdb_db_write_micros_count',
       'rocksdb_db_write_micros_sum', 'rocksdb_db_seek_micros_p50',
       'rocksdb_db_seek_micros_count', 'rocksdb_db_seek_micros_sum',
       'rocksdb_bytes_per_read_count

In [30]:
dataset_y.sort_values(by='rocksdb_bytes_per_read_count')

,cpu_p99,mem_p99,exe_time,mixgraph_latency,mixgraph_iops,mixgraph_io_size,FileReadStatsLevel0_level,FileReadStatsLevel0_count,FileReadStatsLevel0_average,FileReadStatsLevel0_stddev,...,level_1_moved_gb,level_1_w_amp,level_1_rd_mb_per_s,level_1_wr_mb_per_s,level_1_comp_sec,level_1_comp_merge_cpu_sec,level_1_comp_cnt,level_1_avg_sec,level_1_key_in,level_1_key_drop
272,35.444,0.224998,300.317647,249.502,4007.0,2.9,0.0,11048.0,786.7051,386.00,...,0.0,6.5,14.6,14.6,4.86,3.95,10.0,0.486,0.934,1657.0
127,37.946,0.209712,310.297828,227.875,4388.0,3.2,0.0,14136.0,824.6251,405.06,...,0.0,6.0,14.7,14.7,4.97,3.89,9.0,0.553,0.962,2045.0
472,100.100,0.065867,310.254411,167.532,5969.0,4.5,0.0,869.0,1141.7595,265.99,...,0.0,1.0,23.1,23.3,0.23,0.12,1.0,0.233,0.148,1572.0
272,100.213,0.057164,310.250481,166.881,5992.0,4.5,0.0,873.0,1215.8786,356.61,...,0.0,1.0,23.8,24.1,0.23,0.12,1.0,0.226,0.148,1574.0
118,100.100,0.063746,300.320248,164.949,6062.0,4.6,0.0,880.0,1066.4852,321.00,...,0.0,1.0,19.5,19.7,0.26,0.15,1.0,0.258,0.138,1371.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,116.113,0.423190,310.323385,9.232,108319.0,100.5,0.0,5459.0,1006.7848,284.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,111.602,0.421175,310.326910,9.232,108323.0,100.5,0.0,5587.0,920.8135,285.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109.576,0.415191,300.318501,9.231,108335.0,100.5,0.0,7466.0,989.3378,270.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,105.536,0.413314,300.313284,9.229,108348.0,100.5,0.0,2697.0,1670.1683,341.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
import torch

vals = torch.randn((5, 10))
mask = torch.zeros_like(vals)

vals * (mask + torch.tensor([0,1,0,0,0,1,0,0,0,0]).reshape(1,-1))
# dataset_y['rocksdb_read_block_get_micros_p99']

tensor([[-0.0000,  0.2024,  0.0000,  0.0000, -0.0000,  1.6478,  0.0000, -0.0000,
          0.0000,  0.0000],
        [-0.0000,  1.6429, -0.0000, -0.0000, -0.0000, -1.6431, -0.0000, -0.0000,
          0.0000, -0.0000],
        [ 0.0000,  0.3813, -0.0000, -0.0000,  0.0000, -0.4932,  0.0000, -0.0000,
          0.0000,  0.0000],
        [-0.0000, -0.2194, -0.0000, -0.0000,  0.0000,  1.0457,  0.0000,  0.0000,
          0.0000, -0.0000],
        [-0.0000, -1.5892, -0.0000, -0.0000,  0.0000,  0.3837, -0.0000, -0.0000,
          0.0000, -0.0000]])